##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aproximación de matrices con Core API

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/matrix_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/matrix_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/matrix_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver el código fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/core/matrix_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a> </td>
</table>

## Introducción

Este bloc de notas utiliza las API de [bajo nivel de TensorFlow Core](https://www.tensorflow.org/guide/core) para mostrar las funciones de TensorFlow como plataforma de computación científica de alto rendimiento. Visite la [Descripción general de las API del núcleo](https://www.tensorflow.org/guide/core) para obtener más información sobre TensorFlow Core y sus casos de uso específicos.

Este tutorial explora la técnica de [descomposición de valor singular](https://developers.google.com/machine-learning/recommendation/collaborative/matrix) (SVD) y sus aplicaciones para problemas de aproximación de bajo rango. La SVD se utiliza para factorizar matrices reales o complejas y tiene diversos usos en la ciencia de datos, como la compresión de imágenes. Las imágenes para este tutorial provienen del proyecto [Imagen](https://imagen.research.google/) de Google Brain. 

> ![svd_intro](http://tensorflow.org/images/core/svd_intro.png)

## Preparación

In [ ]:
import matplotlib
from matplotlib.image import imread
from matplotlib import pyplot as plt
import requests
# Tamaños de figuras de Matplotlib preestablecidos.
matplotlib.rcParams['figure.figsize'] = [16, 9]

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Fundamentos de la SVD

La descomposición en valores singulares de una matriz, ${\mathrm{A}}$, se determina por la siguiente factorización:

$${\mathrm{A}} = {\mathrm{U}} \Sigma {\mathrm{V}}^T$$

donde:

- $\underset{m \times n}{\mathrm{A}}$: matriz de entrada donde $m \geq n$
- $\underset{m \times n}{\mathrm{U}}$: matriz ortogonal, ${\mathrm{U}}^T{\mathrm{U}} = {\mathrm{I}}$, con cada columna, $u_i$, que denota un vector singular izquierdo de ${\mathrm{A}}$
- $\underset{n \times n}{\Sigma}$: matriz diagonal con cada entrada diagonal, $\sigma_i$, que denota un valor singular de ${\mathrm{A}}$
- $\underset{n \times n}{{\mathrm{V}}^T}$: matriz ortogonal, ${\mathrm{V}}^T{\mathrm{V}} = {\mathrm{I}}$, con cada fila, $v_i$, que denota un vector recto singular de ${\mathrm{A}}$

Cuando tanto $m < n$, ${\mathrm{U}}$ como $\Sigma$ tienen dimensión $(m \times m)$, y ${\mathrm{V}}^T$ tiene la dimensión $(m \times n)$.

> ![svd_full](http://tensorflow.org/images/core/svd_full.png)

El paquete de álgebra lineal de TensorFlow tiene una función, `tf.linalg.svd`, que se puede utilizar para calcular la descomposición del valor singular de una o más matrices. Comience por definir una matriz simple y calcular su factorización SVD.


In [ ]:
A = tf.random.uniform(shape=[40,30])
# Calcula la factorización SVD
s, U, V = tf.linalg.svd(A)
# Define Sigma y V Transpose
S = tf.linalg.diag(s)
V_T = tf.transpose(V)
# Reconstruye la matriz original
A_svd = U@S@V_T
# Visualizar 
plt.bar(range(len(s)), s);
plt.xlabel("Singular value rank")
plt.ylabel("Singular value")
plt.title("Bar graph of singular values");

La función `tf.einsum` puede utilizarse para calcular directamente la reconstrucción de la matriz a partir de las salidas de `tf.linalg.svd`.

In [ ]:
A_svd = tf.einsum('s,us,vs -> uv',s,U,V)
print('\nReconstructed Matrix, A_svd', A_svd)

## Aproximación de bajo rango con la SVD

El rango de una matriz, ${\mathrm{A}}$, se determina por la dimensión del espacio vectorial que abarcan sus columnas. La SVD puede utilizarse para aproximar una matriz con un rango inferior, lo que en última instancia disminuye la dimensionalidad de los datos necesarios para almacenar la información que representa la matriz.

La aproximación de rango-r de ${\mathrm{A}}$ en términos de la SVD se define por la fórmula:

$${\mathrm{A_r}} = {\mathrm{U_r}} \Sigma_r {\mathrm{V_r}}^T$$

donde:

- $\underset{m \times r}{\mathrm{U_r}}$: es una matriz formada por las primeras $r$ columnas de ${\mathrm{U}}$
- $\underset{r \times r}{\Sigma_r}$: es una matriz diagonal formada por los primeros $r$ valores singulares en $\Sigma$
- $\underset{r \times n}{\mathrm{V_r}}^T$: es una matriz formada por las primeras $r$ filas de ${\mathrm{V}}^T$

> ![svd_approx](http://tensorflow.org/images/core/svd_approx.png)

Comience a escribir una función para calcular la aproximación de rango-r de una matriz dada. Este procedimiento de aproximación de bajo rango se utiliza para la compresión de imágenes; por lo tanto, también es útil calcular el tamaño físico de los datos para cada aproximación. Para simplificar, supongamos que el tamaño de los datos para una matriz aproximada de rango-r es igual al número total de elementos necesarios para calcular la aproximación. A continuación, escriba una función para visualizar la matriz original, $\mathrm{A}$ su aproximación de rango-r, $\mathrm{A}_r$ y la matriz de error, $||mathrm{A} - \mathrm{A}_r|$.

In [ ]:
def rank_r_approx(s, U, V, r, verbose=False):
  # Calcula las matrices necesarias para una aproximación rank-r
  s_r, U_r, V_r = s[..., :r], U[..., :, :r], V[..., :, :r] # ... implica cualquier número de ejes de lotes extra
  # Calcular la aproximación de bajo rango y su tamaño
  A_r = tf.einsum('...s,...us,...vs->...uv',s_r,U_r,V_r)
  A_r_size = tf.size(U_r) + tf.size(s_r) + tf.size(V_r)
  if verbose:
    print(f"Approximation Size: {A_r_size}")
  return A_r, A_r_size

def viz_approx(A, A_r):
# Grafica A, A_r, y A - A_r
  vmin, vmax = 0, tf.reduce_max(A)
  fig, ax = plt.subplots(1,3)
  mats = [A, A_r, abs(A - A_r)]
  titles = ['Original A', 'Approximated A_r', 'Error |A - A_r|']
  for i, (mat, title) in enumerate(zip(mats, titles)):
    ax[i].pcolormesh(mat, vmin=vmin, vmax=vmax)
    ax[i].set_title(title)
    ax[i].axis('off')

In [ ]:
print(f"Original Size of A: {tf.size(A)}")
s, U, V = tf.linalg.svd(A)

In [ ]:
# Aproximación de rango-15
A_15, A_15_size = rank_r_approx(s, U, V, 15, verbose = True)
viz_approx(A, A_15)

In [ ]:
# Aproximación de rango-3
A_3, A_3_size = rank_r_approx(s, U, V, 3, verbose = True)
viz_approx(A, A_3)

Como era de suponer, el uso de rangos más bajos produce aproximaciones menos precisas. Sin embargo, la calidad de estas aproximaciones de bajo rango a menudo es suficientemente buena en situaciones reales. También hay se debe tener en cuenta que el objetivo principal de la aproximación de bajo rango con SVD es reducir la dimensionalidad de los datos, pero no reducir el espacio en disco de los propios datos. Sin embargo, conforme las matrices de entrada adquieren mayor dimensionalidad, muchas aproximaciones de bajo rango también acaban beneficiándose de la reducción del tamaño de los datos. Este beneficio de reducción es la razón por la que el proceso es útil para los problemas de compresión de imágenes.

## Cargando imágenes

La siguiente imagen está disponible en la página de inicio de [Imagen](https://imagen.research.google/). Imagen es un modelo de difusión de texto a imagen desarrollado por el equipo Brain de Google Research. Una IA creó esta imagen a partir de la pregunta: "Una foto de un perro Corgi montando en bicicleta en Times Square. Lleva gafas de sol y un sombrero de playa". ¡Qué maravilla! También puedes cambiar la url que aparece a continuación por cualquier enlace .jpg para cargar una imagen personalizada de tu elección.

Comienza leyendo y visualizando la imagen. Después de leer un archivo JPEG, Matplotlib produce una matriz, ${\mathrm{I}}$, de forma $(m \times n \times 3)$ que representa una imagen bidimensional con 3 canales de color para el rojo, verde y azul respectivamente.

In [ ]:
img_link = "https://imagen.research.google/main_gallery_images/a-photo-of-a-corgi-dog-riding-a-bike-in-times-square.jpg"
img_path = requests.get(img_link, stream=True).raw
I = imread(img_path, 0)
print("Input Image Shape:", I.shape)

In [ ]:
def show_img(I):
  # Muestra la imagen en matplotlib
  img = plt.imshow(I)
  plt.axis('off')
  return

In [ ]:
show_img(I)

## El algoritmo de compresión de imágenes

Ahora, utilice la SVD para calcular aproximaciones de bajo rango para la imagen de muestra. Recuerde que la imagen es de forma $(1024 \times 1024 \times 3)$ y que la teoría SVD solo se aplica para matrices bidimensionales. Esto significa que la imagen de muestra debe dividirse en 3 matrices del mismo tamaño para cada uno de los 3 canales de color. Esto se puede hacer mediante la transposición de la matriz para que tenga la forma $(3 \times 1024 \times 1024)$. Con el fin de visualizar claramente el error de aproximación, reescalar los valores RGB de la imagen de $[0,255]$ a $[0,1]$. Recuerde que debe recortar los valores aproximados para que caigan dentro de este intervalo antes de visualizarlos. La función `tf.clip_by_value` es útil para este propósito.

In [ ]:
def compress_image(I, r, verbose=False):
# Comprime una imagen con la SVD con un rango determinado 
  I_size = tf.size(I)
  print(f"Original size of image: {I_size}")
  # Compute SVD of image
  I = tf.convert_to_tensor(I)/255
  I_batched = tf.transpose(I, [2, 0, 1]) # einops.rearrange(I, 'h w c -> c h w')
  s, U, V = tf.linalg.svd(I_batched)
  # Calcula la aproximación de bajo rango de la imagen en cada canal RGB
  I_r, I_r_size = rank_r_approx(s, U, V, r)
  I_r = tf.transpose(I_r, [1, 2, 0]) # einops.rearrange(I_r, 'c h w -> h w c')
  I_r_prop = (I_r_size / I_size)
  if verbose:
# Muestra la imagen comprimida y sus atributos
    print(f"Number of singular values used in compression: {r}")
    print(f"Compressed image size: {I_r_size}")
    print(f"Proportion of original size: {I_r_prop:.3f}")
    ax_1 = plt.subplot(1,2,1)
    show_img(tf.clip_by_value(I_r,0.,1.))
    ax_1.set_title("Approximated image")
    ax_2 = plt.subplot(1,2,2)
    show_img(tf.clip_by_value(0.5+abs(I-I_r),0.,1.))
    ax_2.set_title("Error")
  return I_r, I_r_prop

Ahora, calcule aproximaciones de rango-r para los siguientes rangos: 100, 50, 10

In [ ]:
I_100, I_100_prop = compress_image(I, 100, verbose=True)

In [ ]:
I_50, I_50_prop = compress_image(I, 50, verbose=True)

In [ ]:
I_10, I_10_prop = compress_image(I, 10, verbose=True)

## Evaluación de las aproximaciones

Hay una variedad de métodos interesantes para medir la eficacia y tener más control sobre las aproximaciones de las matrices.

### Factor de compresión vs rango

Para cada una de las aproximaciones anteriores, observe cómo cambia el tamaño de los datos en función del rango.

In [ ]:
plt.figure(figsize=(11,6))
plt.plot([100, 50, 10], [I_100_prop, I_50_prop, I_10_prop])
plt.xlabel("Rank")
plt.ylabel("Proportion of original image size")
plt.title("Compression factor vs rank");

Según este gráfico, hay una relación lineal entre el factor de compresión de una imagen aproximada y su rango. Para profundizar en este tema, recordemos que el tamaño de los datos de una matriz aproximada, ${mathrm{A}}_r$, se define como el número de elementos totales necesarios para calcularla. Las siguientes ecuaciones se pueden utilizar para encontrar la relación entre el factor de compresión y el rango:

$$x = (m \times r) + r + (r \times n) = r \times (m + n + 1)$$

$$c = \large \frac{x}{y} = \frac{r \times (m + n + 1)}{m \times n}$$

donde:

- $x$: tamaño de ${\mathrm{A_r}}$
- $y$: tamaño de ${\mathrm{A}}$
- $c = \frac{x}{y}$: factor de compresión
- $r$: rango de la aproximación
- $m$ y $n$: dimensiones de la fila y la columna de ${\mathrm{A}}$

Para encontrar el rango, $r$, que es necesario para comprimir una imagen en un factor deseado, $c$, la ecuación anterior se puede reorganizar para solucionar $r$:

$$r = ⌊{\large\frac{c \times m \times n}{m + n + 1}}⌋$$

Tenga en cuenta que esta fórmula es independiente del canal de color, ya que cada una de las aproximaciones RGB no se modifican entre sí. Ahora, escriba una función para comprimir una imagen de entrada dada por un factor de compresión deseado.

In [ ]:
def compress_image_with_factor(I, compression_factor, verbose=False):
  # Devuelve una imagen comprimida basada en un factor de compresión deseado
  m,n,o = I.shape
  r = int((compression_factor * m * n)/(m + n + 1))
  I_r, I_r_prop = compress_image(I, r, verbose=verbose)
  return I_r

Comprima una imagen hasta un 15% de su tamaño original.

In [ ]:
compression_factor = 0.15
I_r_img = compress_image_with_factor(I, compression_factor, verbose=True)

### Suma acumulada de valores únicos

La suma acumulada de valores únicos puede ser un indicador útil para la cantidad de energía capturada por una aproximación rank-r. Visualice la proporción acumulada promedio de valores únicos RGB en la imagen de muestra. La función `tf.cumsum` puede ser útil para esto.

In [ ]:
def viz_energy(I):
# Visualice la energía capturada basada en el rango
# Cálculo de la SVD
  I = tf.convert_to_tensor(I)/255
  I_batched = tf.transpose(I, [2, 0, 1]) 
  s, U, V = tf.linalg.svd(I_batched)
# Graficar la proporción promedio mediante los canales RGB 
  props_rgb = tf.map_fn(lambda x: tf.cumsum(x)/tf.reduce_sum(x), s)
  props_rgb_mean = tf.reduce_mean(props_rgb, axis=0)
  plt.figure(figsize=(11,6))
  plt.plot(range(len(I)), props_rgb_mean, color='k')
  plt.xlabel("Rank / singular value number")
  plt.ylabel("Cumulative proportion of singular values")
  plt.title("RGB-averaged proportion of energy captured by the first 'r' singular values")

In [ ]:
viz_energy(I)

Parece que más del 90% de la energía de esta imagen se captura dentro de los 100 primeros valores únicos. Ahora, escriba una función para comprimir una imagen de entrada con el factor de retención de energía que desee.

In [ ]:
def compress_image_with_energy(I, energy_factor, verbose=False):
  # Devuelve una imagen comprimida basada en un factor de energía deseado
# Calcula la SVD
  I_rescaled = tf.convert_to_tensor(I)/255
  I_batched = tf.transpose(I_rescaled, [2, 0, 1]) 
  s, U, V = tf.linalg.svd(I_batched)
# Extracción de valores únicos
  props_rgb = tf.map_fn(lambda x: tf.cumsum(x)/tf.reduce_sum(x), s)
  props_rgb_mean = tf.reduce_mean(props_rgb, axis=0)
# Encuentra la r más cercana que corresponde al factor de energía
  r = tf.argmin(tf.abs(props_rgb_mean - energy_factor)) + 1
  actual_ef = props_rgb_mean[r]
  I_r, I_r_prop = compress_image(I, r, verbose=verbose)
  print(f"Proportion of energy captured by the first {r} singular values: {actual_ef:.3f}")
  return I_r

Comprime una imagen para retener el 75% de su energía.

In [ ]:
energy_factor = 0.75
I_r_img = compress_image_with_energy(I, energy_factor, verbose=True)

### Error y valores únicos

También hay una relación interesante entre el error de aproximación y los valores únicos. Resulta que la norma de Frobenius al cuadrado de la aproximación es igual a la suma de los cuadrados de sus valores únicos excluidos:

$${||A - A_r||}^2 = \sum_{i=r+1}^{R}σ_i^2$$

Pruebe esta relación con una aproximación de rango 10 de la matriz de ejemplo que aparece al principio de este tutorial.

In [ ]:
s, U, V = tf.linalg.svd(A)
A_10, A_10_size = rank_r_approx(s, U, V, 10)
squared_norm = tf.norm(A - A_10)**2
s_squared_sum = tf.reduce_sum(s[10:]**2)
print(f"Squared Frobenius norm: {squared_norm:.3f}")
print(f"Sum of squared singular values left out: {s_squared_sum:.3f}")

## Conclusión

En este bloc de notas se introdujo el proceso de implementación de la descomposición de valor único con TensorFlow y su aplicación para escribir un algoritmo de compresión de imágenes. Aquí encontrará algunos consejos adicionales que pueden ser útiles:

- Las [API de TensorFlow Core](https://www.tensorflow.org/guide/core) se pueden utilizar para resolver una gran variedad de casos de computación científica de alto rendimiento.
- Para obtener más información sobre las funciones de álgebra lineal de TensorFlow, visite la documentación del [módulo linalg](https://www.tensorflow.org/api_docs/python/tf/linalg).
- La SVD también puede aplicarse para crear [sistemas de recomendación](https://developers.google.com/machine-learning/recommendation/labs/movie-rec-programming-exercise).

Para obtener más ejemplos sobre el uso de las API de TensorFlow Core, consulte la [guía](https://www.tensorflow.org/guide/core). Si desea obtener más información sobre la carga y preparación de datos, consulte los tutoriales sobre la [carga de datos de imagen](https://www.tensorflow.org/tutorials/load_data/images) o la [carga de datos CSV](https://www.tensorflow.org/tutorials/load_data/csv).